In [1]:
import pandas as pd
import numpy as np
import transformers as ppb # pytorch transformers
from sklearn.model_selection import train_test_split
import torch
import math

In [2]:
df = pd.read_csv('datasets/toxic_comments.csv')

In [3]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Загрузка предобученной модели/токенизатора 
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# Сделаем токенизацию
tokenized = df['text'].apply((lambda x: tokenizer.encode(x[:512], add_special_tokens=True)))

In [5]:
# Найдём максимальную длину списка
max_len = len(max(tokenized, key=len))

# Приведем весь список к одинаковой длине
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

In [6]:
input_ids = torch.tensor(padded)

In [7]:
input_ids.shape

torch.Size([159292, 493])

In [27]:
batch_size = 512
embeddings = []
for i in range(math.ceil(padded.shape[0] / batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())    


: 

: 